In [2]:
import math
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from geant3_parser import build_train_set
# load the dataset
from geant3_parser import Geant3DataFile
import os

def get_bin_centers(bins):
    assert len(bins) >= 2
    return bins[:-1] + (bins[1:] - bins[:-1]) / 2

def histo_rms(histo, bins):
    bin_centers = get_bin_centers(bins)

    rms = np.sum(histo*(bin_centers*bin_centers))/np.sum(histo)
    rms = math.sqrt(rms)
    return rms

max_cell_index = 5
total_columns = max_cell_index * 2 + 1
total_rows = total_columns              # square calorimeter

file_name = os.path.join('data', 'shower_geant3_new.dat')
# file_name = 'sample_data.txt'

data_file = Geant3DataFile(file_name, skip_lines=3)


model = load_model('g3_dense_with_xy.hd5')



Time open 0.6452319622039795
Time split 0.30503010749816895


In [3]:
len(build_train_set(data_file, 5000, add_real_xy=True))

(array([[0.16288722, 0.32826526, 0.6722825 , ..., 0.        , 0.        ,
         0.18904014],
        [0.29619059, 0.47652246, 0.72833516, ..., 0.        , 0.        ,
         0.06301338],
        [0.06301338, 0.24618638, 0.50375123, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.36430302, 0.5979874 , 0.7071482 , ..., 0.        , 0.        ,
         0.14631254],
        [0.4731    , 0.12602676, 0.56219535, ..., 0.        , 0.        ,
         0.        ],
        [0.3897    , 0.31218065, 0.69319156, ..., 0.        , 0.        ,
         0.16288722]]),
 array([[2.3556],
        [4.4761],
        [0.3179],
        ...,
        [4.0379],
        [1.4192],
        [3.3711]]),
 array([[2265.],
        [4173.],
        [ 311.],
        ...,
        [3791.],
        [1322.],
        [3249.]]))

In [ ]:

input_data, true_e, reco_e, sum_e = build_train_set(data_file, 50000, add_real_xy=True)

eval_e = model.predict(input_data)
eval_e = eval_e.flatten()
reco_e = reco_e.flatten()
true_e = true_e.flatten()
sum_e = sum_e.flatten()

plt.figure(figsize=(10, 10))
plt.hist2d(true_e, eval_e, bins=100, range=[[0, 6], [0, 6]])
plt.show()


eval_dee = np.sqrt(np.power((true_e - eval_e), 2)) / true_e
mask = np.isfinite(eval_dee)
eval_dee = eval_dee[mask]
true_e = true_e[mask]
reco_e = reco_e[mask]
eval_e = eval_e[mask]
del mask

plt.figure(figsize=(10, 10))
plt.hist2d(true_e, eval_dee, bins=100, range=[[0, 6], [0, 0.1]])
plt.show()

err_bins = np.arange(0, 0.1, 0.005)
e_bins = np.arange(0, 5, 0.1)

eval_dee_hist, _, _ = np.histogram2d(true_e, eval_dee, bins=[e_bins, err_bins])


e_bin_centers = get_bin_centers(e_bins)
eval_rmses=[histo_rms(h1d, err_bins) for h1d in eval_dee_hist]

fig = plt.figure(figsize=(10, 8), dpi=150)
ax = fig.add_subplot(111)
ax.plot(e_bin_centers, eval_rmses, 'bo')
ax.set_title('dE/E as a function of E for caloML reconstruction')
ax.set_xlabel('E, GeV')
ax.set_ylabel('dE/E')


del eval_dee, eval_dee_hist, eval_e


# reco_dee = np.sqrt(np.power((true_e - reco_e), 2)) / true_e
# reco_dee_hist, _, _ = np.histogram2d(true_e, reco_dee, bins=[e_bins, err_bins])
# reco_rmses=[histo_rms(h1d, err_bins) for h1d in reco_dee_hist]
#
# plt.plot(e_bin_centers, reco_rmses, 'ro--')
plt.show()
